# MOSCOW AIRPORTS PARSE

In [1]:
#Импортируем библиотеки для парсинга
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import time

#Импортируем библиотеки для работы с данными
import pandas as pd

#Доп библиотека для столбца со временем выгрузки
import datetime

In [2]:
#Установка Selenium (по необходимости)
#!pip install selenium

### 1. SHEREMETIEVO PARSE (SVO)

In [3]:
#Подключаемся к сайту Аэропорта Шереметьево (онлайн-табло на текущий день) с помошью Selenium
driver_svo = webdriver.Chrome()
driver_svo.get("URL") 

#указываем условие для прокрутки сайта в автоматическом режиме до самого конца
lenOfPage = driver_svo.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    time.sleep(3)
    lenOfPage = driver_svo.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount==lenOfPage:
        match=True

#создадим пустые списки для заполнения данными с сайта
data_city_svo=[]
data_airlines_svo=[]

#создадим пустые списки для мэппинга для заполнения данными с сайта
data_airlines_codes_svo=[]
data_flight_numbers_svo=[]

#получаем данные по названию города
city_names_svo = driver_svo.find_elements(By.CLASS_NAME, "flight-row__city-name")

#получаем данные по названию авиакомпании (по имени логотипа)
airlines_2_svo=[]
airlines_svo = driver_svo.find_elements(By.TAG_NAME, "img")
for i in airlines_svo:
    airlines_2_svo.append(i.get_attribute("alt"))
    
#получаем данные по номеру рейса (для мэппинга)
flight_numbers_svo = driver_svo.find_elements(By.CLASS_NAME, "flight-row__airline-flight-number")
#получаем данные по коду АК(для мэппинга)
airlines_codes_svo = driver_svo.find_elements(By.CLASS_NAME, "flight-row__airline-name")

#заполняем пустые списки, указанные выше данными с сайта
for city in city_names_svo:
    data_city_svo.append(city.text)
for airline in airlines_2_svo:
    data_airlines_svo.append(airline)

#заполняем пустые списки, указанные выше данными с сайта (для мэппинга)
for numbers in flight_numbers_svo:
    data_airlines_codes_svo.append(numbers.text)
for codes in airlines_codes_svo:
    data_flight_numbers_svo.append(codes.text)

driver_svo.close()

In [4]:
#склеиваем датафрейм
data_city_svo=pd.Series(data_city_svo)
data_airlines_svo=pd.Series(data_airlines_svo)
df_1_svo = pd.concat([data_city_svo, data_airlines_svo], axis=1)
#переименовываем столбцы
df_1_svo.columns=['city', 'airline']
#убираем лишные данные
df_SVO = df_1_svo[~df_1_svo['city'].isna()][['city', 'airline']]
#финальный датафрейм
df_SVO=df_SVO.reset_index()

In [5]:
#создаем датафрейм из двух столбцов для создания в конечном итоге столбца без дублей по кодам IATA (на сайте иногда указаны два кода
# первый - кто продал билет - 2 перевозчик. Мы берем первый код)
data_airlines_codes_svo=pd.Series(data_airlines_codes_svo)
data_flight_numbers_svo=pd.Series(data_flight_numbers_svo)
df_mapping_svo = pd.concat([data_airlines_codes_svo, data_flight_numbers_svo], axis=1)
df_mapping_svo.columns=['number','code']

In [6]:
#убираем ненужные строки, чтоб длина столбца с кодами = длине столбца с навазниями компаний и городами (df_SVO)
df_mapping_svo['new'] = df_mapping_svo['number'].shift(1)
#промежуточная функция (лучше не придумал)
def dropp (df):
    if df['number'] == df['new']:
        return df['code'] 
    else:
        return df['number']
#промежуточная функция (лучше не придумал)
def dropp_2 (df):
    if df['number'] == df['new_2']:
        return df['number']
    else:
        return 0
#применяем функции и получаем датафрейм
df_mapping_svo['new_2'] = df_mapping_svo.apply(dropp, axis=1)
df_mapping_svo['new_3'] = df_mapping_svo.apply(dropp_2, axis=1)
df_mapping_svo = df_mapping_svo['code'][df_mapping_svo['new_3'] != 0].reset_index()

In [7]:
#создаем финальный датафрейм (не забываем reset_index перед concat, иначе будут пустые значения появляться)
df_final_svo = pd.concat([df_SVO , df_mapping_svo], axis = 1)
df_final_svo = df_final_svo[['city', 'airline', 'code']]
df_final_svo['airport'] = 'SVO'
now = datetime.datetime.now()
df_final_svo['created_at'] = now

In [8]:
#загружаем файл для единого мэппинга навзваний авиакомпаний по коду ИАТА
mapping_codes_AVS = pd.read_excel(r'C:\Users\MacBook\AVS_mapping_IATA_code_Airline_name.xlsx') 
mapping_codes_AVS = mapping_codes_AVS[['airline_code','airline_name']]
mapping_codes_AVS = mapping_codes_AVS.drop_duplicates()
mapping_codes_AVS = mapping_codes_AVS[mapping_codes_AVS['airline_code'] != mapping_codes_AVS['airline_name']]

In [9]:
#подтягиваем названия авиакомпаний и получаем итоговый датафрейм
df_final_svo = pd.merge(df_final_svo, mapping_codes_AVS, left_on='code', right_on='airline_code', how='left')
df_final_svo = df_final_svo[['city','code','airline_name','airport','created_at']]
df_final_svo.columns=('city','airline_code','airline_name','airport','created_at')
df_final_svo

,city,airline_code,airline_name,airport,created_at
0,Горно-Алтайск,SU,Aeroflot,SVO,2023-01-24 18:49:47.515269
1,Кайо-Коко,N4,NordwindAirlines,SVO,2023-01-24 18:49:47.515269
2,Самарканд,SU,Aeroflot,SVO,2023-01-24 18:49:47.515269
3,Омск,SU,Aeroflot,SVO,2023-01-24 18:49:47.515269
4,Екатеринбург,DP,Pobeda,SVO,2023-01-24 18:49:47.515269
...,...,...,...,...,...
242,Омск,DP,Pobeda,SVO,2023-01-24 18:49:47.515269
243,Тегеран,W5,MahanAirlines,SVO,2023-01-24 18:49:47.515269
244,Томск,SU,Aeroflot,SVO,2023-01-24 18:49:47.515269
245,Красноярск,SU,Aeroflot,SVO,2023-01-24 18:49:47.515269


### 2. DOMODEDOVO PARSE (DME)

In [10]:
#подключаемся к сайту через selenium
driver_dme = webdriver.Chrome()
driver_dme.get("URL")

#указываем условие для прокрутки сайта в автоматическом режиме до самого конца
lenOfPage = driver_dme.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    time.sleep(3)
    lenOfPage = driver_dme.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount==lenOfPage:
        match=True

#получаем с сайта строки из таблицы с онлайн-рейсами
strings_all_dme=[]
strings_dme = driver_dme.find_elements(By.CLASS_NAME, 'vat ')
for st in strings_dme:
    strings_all_dme.append(st.text.split('\n'))

driver_dme.close()

In [11]:
#создаем датафрейм из строк в таблице
df_dme = pd.DataFrame(strings_all_dme)

In [12]:
#усдвигаем строки, в которые в первый столбец попали разделители по часам

df_dme.replace('▼', '', inplace=True, regex=True)
df_dme.replace('▲', '', inplace=True, regex=True)

spisok_time_dme = ['01:00','02:00','03:00','04:00','05:00','06:00','07:00','08:00','09:00','10:00','11:00','12:00','13:00','14:00', '15:00','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00','▼23:00', '▼24:00','24:00']

for i in spisok_time_dme:
    m = (df_dme[0] == i)
    df_dme.loc[m]  = df_dme.loc[m].shift(-1, axis=1)


In [13]:
#удаляем все нелатинские ячейки в датафрейме
def sh_2 (df):
    if df[2].isascii():
        return df.drop(2)
    else:
        return df
df_dme = df_dme.apply(sh_2, axis=1)

In [14]:
#df_dme.loc[df_dme[3].str.contains('7', regex=True)] = df_dme.loc[df_dme[3].str.contains('7', regex=True)].shift(-1, axis=1)
#df_dme.tail(50)

In [15]:
#заменяем все образовывшиеся пустоты  на название горда (т.к. латинское обозначение - это обозначение второй авиакомпании, которую мы не используем)
df_dme[2] = df_dme[2].fillna(df_dme[3])

In [16]:
#заменяем все нелатинские ячейки в столбце с городом в датафрейме
def sh_3 (df):
    if df[2].isascii():
        return df[4]
    else:
        return df[2]
df_dme[2] = df_dme.apply(sh_3, axis=1)

In [17]:
#выносим код авиакомпании в отдельный столбце и оставляем только нужные столбцы
df_2_dme = df_dme[1].str.split(' ', expand=True)
df_2_dme=df_2_dme[[0,1]]
df_2_dme.columns = ('airline_code','flight_number')
df_final_dme = pd.concat([df_dme, df_2_dme], axis=1)
#df_final = df_final[[1,2]]
df_final_dme=df_final_dme[[2,'airline_code']]
df_final_dme.columns=('city', 'airline_code')


In [18]:
#загружаем файл для единого мэппинга навзваний авиакомпаний по коду ИАТА
mapping_codes_AVS = pd.read_excel(r'C:\Users\MacBook\AVS_mapping_IATA_code_Airline_name.xlsx') 
mapping_codes_AVS = mapping_codes_AVS[['airline_code','airline_name']]
mapping_codes_AVS = mapping_codes_AVS.drop_duplicates()
mapping_codes_AVS = mapping_codes_AVS[mapping_codes_AVS['airline_code'] != mapping_codes_AVS['airline_name']]

In [19]:
#подтягиваем названия авиакомпаний и получаем итоговый датафрейм
df_final_dme = pd.merge(df_final_dme, mapping_codes_AVS, left_on='airline_code', right_on='airline_code', how='left')
df_final_dme['airport'] = 'DME'
now = datetime.datetime.now()
df_final_dme['created_at'] = now
df_final_dme

,city,airline_code,airline_name,airport,created_at
0,УФА,S7,S7Airlines,DME,2023-01-24 18:50:23.908694
1,МУРМАНСК,S7,S7Airlines,DME,2023-01-24 18:50:23.908694
2,ШАРДЖА,G9,AirArabia,DME,2023-01-24 18:50:23.908694
3,КЫЗЫЛ,IO,IrAero,DME,2023-01-24 18:50:23.908694
4,КУЛЯБ,U6,UralAirlines,DME,2023-01-24 18:50:23.908694
...,...,...,...,...,...
144,ОШ,U6,UralAirlines,DME,2023-01-24 18:50:23.908694
145,БАКУ,J2,AzerbaijanAirlines(Azal),DME,2023-01-24 18:50:23.908694
146,ДУБАЙ,EK,Emirates,DME,2023-01-24 18:50:23.908694
147,НОРИЛЬСК,Y7,NordStar,DME,2023-01-24 18:50:23.908694


### 3. VNUKOVO PARSE (VKO)

In [20]:
#подключаемся к сайту через selenium
driver_vko = webdriver.Chrome()
driver_vko.get("URL") 
time.sleep(3)

#нажимаем на кнопку "вчера"
clickable  = driver_vko.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[3]/div[4]/div[2]/form/div[1]/div/a[2]')
time.sleep(3)
ActionChains(driver_vko).click(clickable).perform()
time.sleep(3)


#указываем условие для прокрутки сайта в автоматическом режиме до самого конца
lenOfPage = driver_vko.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    time.sleep(3)
    lenOfPage = driver_vko.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount==lenOfPage:
        match=True

#получаем с сайта строки из таблицы с онлайн-рейсами
strings_all_vko=[]
strings_vko = driver_vko.find_elements(By.TAG_NAME, 'tr')
for st in strings_vko:
    strings_all_vko.append(st.text.split('\n'))
    
driver_vko.close()

In [21]:
#создаем датафрейм из полученных строк
df_vko = pd.DataFrame(strings_all_vko)
#убираем все пустые строки
df_vko = df_vko.dropna()
df_vko

,0,1,2,3
0,00:05,23.01.2023 РГ 4351,РусЛайн,Ханты-Мансийск А Совершил посадку 03:00
1,00:05,23.01.2023 ЮТ 351,Ютэйр,Ханты-Мансийск А Совершил посадку 03:00
2,00:20,23.01.2023 РГ 4247,РусЛайн,Сургут А Совершил посадку 03:41
3,00:20,23.01.2023 ДР 427,Победа,Оренбург (Центральный им.Гагарина) А Совершил ...
4,00:20,23.01.2023 ЮТ 247,Ютэйр,Сургут А Совершил посадку 03:41
...,...,...,...,...
121,22:40,23.01.2023 DV 816,SCAT,Алматы А Совершил посадку 24.01 03:06
122,23:05,23.01.2023 ZF 1611,АЗУР ЭЙР,Коломбо (Бандаранайке интерн.) А Совершил поса...
123,23:35,23.01.2023 ДР 433,Победа,Пермь (Большое Савино) А Совершил посадку 24.0...
124,23:35,23.01.2023 ДР 435,Победа,Уфа А Совершил посадку 24.01 01:41


In [22]:
#выделяем отдельно код авиакомпании
df_2_vko = df_vko[1].str.split(' ', expand = True)
df_2_vko.columns=('date', 'airline_code', 'flight_number', 'none')
#присоединяем код авиакомпании
df_vko = pd.concat([df_vko, df_2_vko], axis = 1)
df_vko= df_vko[['airline_code', 2, 3]]
#выделим отдельно город
df_3_vko= df_vko[3]. str.split(' ', expand = True)
df_3_vko=df_3_vko.rename(columns={0:'city'})
#присоединяем город
df_vko = pd.concat([df_vko, df_3_vko['city']], axis = 1)
df_vko = df_vko[['airline_code', 2, 'city']]
df_vko.columns = ('airline_code', 'airline_name', 'city')
df_vko = df_vko[['city', 'airline_code','airline_name']]

In [23]:
#подгружаем данные мэппинга РФ кодов и кодов ИАТА (на сайте внуково такая особенность)
rus_iata_mapping = pd.read_csv(r'C:\Users\MacBook\RUS_IATA_mapping_code.csv', sep=';')
rus_iata_mapping = rus_iata_mapping.dropna(subset=['РФ', 'ИАТА'])
rus_iata_mapping = rus_iata_mapping[['ИАТА', 'РФ']]

In [24]:
df_vko=pd.merge(df_vko, rus_iata_mapping, left_on = 'airline_code', right_on='РФ', how='left')

In [25]:
def sh (df):
    if df['airline_code'].isascii() == False:
        return df['ИАТА']
    else:
        return df['airline_code']
df_vko['new_iata_code'] = df_vko.apply(sh, axis=1)

In [26]:
#загружаем файл для единого мэппинга навзваний авиакомпаний по коду ИАТА
mapping_codes_AVS = pd.read_excel(r'C:\Users\MacBook\AVS_mapping_IATA_code_Airline_name.xlsx') 
mapping_codes_AVS = mapping_codes_AVS[['airline_code','airline_name']]
mapping_codes_AVS = mapping_codes_AVS.drop_duplicates()
mapping_codes_AVS = mapping_codes_AVS[mapping_codes_AVS['airline_code'] != mapping_codes_AVS['airline_name']]


In [27]:
#подтягиваем названия авиакомпаний и получаем итоговый датафрейм
df_vko = pd.merge(df_vko, mapping_codes_AVS, left_on='new_iata_code', right_on='airline_code', how='left')
df_vko['airport'] = 'VKO'
now = datetime.datetime.now()
df_vko['created_at'] = now
df_vko = df_vko[['city', 'new_iata_code', 'airline_name_y', 'airport', 'created_at']]
df_vko.columns=('city', 'airline_code', 'airline_name', 'airport', 'created_at')
df_final_vko = df_vko
df_final_vko

,city,airline_code,airline_name,airport,created_at
0,Ханты-Мансийск,7R,RusLine,VKO,2023-01-24 18:51:01.029675
1,Ханты-Мансийск,UT,Utair,VKO,2023-01-24 18:51:01.029675
2,Сургут,7R,RusLine,VKO,2023-01-24 18:51:01.029675
3,Оренбург,DP,Pobeda,VKO,2023-01-24 18:51:01.029675
4,Сургут,UT,Utair,VKO,2023-01-24 18:51:01.029675
...,...,...,...,...,...
121,Алматы,DV,SCAT,VKO,2023-01-24 18:51:01.029675
122,Коломбо,ZF,AzurAir,VKO,2023-01-24 18:51:01.029675
123,Пермь,DP,Pobeda,VKO,2023-01-24 18:51:01.029675
124,Уфа,DP,Pobeda,VKO,2023-01-24 18:51:01.029675


# PIVOT

In [28]:
df_MAU_concat = pd.concat([df_final_svo, df_final_dme, df_final_vko], axis=0)
df_MAU_concat= df_MAU_concat.reset_index()
df_MAU_concat

,index,city,airline_code,airline_name,airport,created_at
0,0,Горно-Алтайск,SU,Aeroflot,SVO,2023-01-24 18:49:47.515269
1,1,Кайо-Коко,N4,NordwindAirlines,SVO,2023-01-24 18:49:47.515269
2,2,Самарканд,SU,Aeroflot,SVO,2023-01-24 18:49:47.515269
3,3,Омск,SU,Aeroflot,SVO,2023-01-24 18:49:47.515269
4,4,Екатеринбург,DP,Pobeda,SVO,2023-01-24 18:49:47.515269
...,...,...,...,...,...,...
517,121,Алматы,DV,SCAT,VKO,2023-01-24 18:51:01.029675
518,122,Коломбо,ZF,AzurAir,VKO,2023-01-24 18:51:01.029675
519,123,Пермь,DP,Pobeda,VKO,2023-01-24 18:51:01.029675
520,124,Уфа,DP,Pobeda,VKO,2023-01-24 18:51:01.029675


In [29]:
#Сохраняем датафрейм в файл эксель
file_name = 'MAU_parsing_{}.xlsx'.format(now.strftime("%d%m%Y_%H%M%S"))
df_MAU_concat.to_excel(file_name)